In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import html5lib
import re
from datetime import datetime

## Get unincorporated area boundaries (Do only once)

from lxml import html 
import time

#Get page of links to each unincorporated area
unincorpurl = 'http://maps.latimes.com/neighborhoods/unincorporated/list/page/1/'
unincorpresponse = requests.get(unincorpurl)
unincorpsoup = BeautifulSoup(unincorpresponse.text, "html.parser")

#Get hyperlinks
alist = unincorpsoup.findAll('a', href=True)

#Loop over each unincorporated area
count = 0
outfile = "data/unincorporated_latimes.txt" #Change to .geojson.  Also fix formatting at a later date
for link in alist:
    if ('Unincorporated' in str(link)):
        print('Working on ', link)
        download_url = 'http://maps.latimes.com/'+ link['href']
        
        pageContent=requests.get(download_url) #Use lxml to be able to scrape javascript.         
        tree = html.fromstring(pageContent.content)
        js=tree.xpath('//*[@id="content"]/div[1]/script/text()') #XPath for the script
        result = re.search('features": \[(.*)]    };    ', str(js[0].replace('\n', ''))) #Extract geoJSON feature
        
        with open(outfile, "a+") as text_file:
            if count < 1 :
                text_file.write('{\n "type": "FeatureCollection", \n "features": [\n') #Untested
            text_file.write(result.group(1)+', \n') #Be sure to go remove the last comma manually and fix the beginning
        time.sleep(1) #pause the code for a sec
        count = count + 1
with open(outfile, "a+") as text_file:
    text_file.write('REMOVE PRECEEDING COMMA \n ] }')
        

#Check file output        
import geopandas as gpd
demog = gpd.read_file('data/unincorporated_latimes.geojson')
demog.plot()

## Daily scrape of LA County Public Health table

In [2]:
#Chose what to scrape
archive = False
titlestring='0527' 

if archive:     
    url = 'data/'+titlestring+'.html' #Uses locally downloaded file from wayback machine to lookup archived site
    soup = BeautifulSoup(open(url),"html.parser")
else:
    url ='http://publichealth.lacounty.gov/media/Coronavirus/locations.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
#    datetime.today().strftime('%Y-%m-%d')
#    titlestring=datetime.today().strftime('%m%d') #Use today's date

In [3]:
tr_elements = soup.find_all('tr', {'class' : 'blue text-white'})[-1].find_next_siblings()

colnames = ['Locations','Total Cases', 'Rate', 'Deaths', 'Death Rate']
#for col in soup.find_all('tr')[0].find_all('td'):
#    colnames.append(col.text.replace('*', ''))
    
df = pd.DataFrame(columns=colnames, index = range(0,len(tr_elements)+2)) #To add in total and Pasadena

row_marker = 0

for row in tr_elements:
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:        
        df.iat[row_marker,column_marker] = column.get_text().replace('--', 'suppressed').replace('City of ', '').replace('Unincorporated - ', '').replace('Los Angeles - ','').replace('*','')
        column_marker += 1
    row_marker+=1

#Add total - currently hardcoded
column_marker =0 
for column in soup.find_all('tr')[5].find_all('td'): #Changed value to 5. Previously 1?

    if (column_marker==0):
        print('Renaming ', column.get_text(), 'to Total')
        df.iat[-1, column_marker] = 'Total'
    else:
        df.iat[-1, column_marker] = column.get_text()
        print(column.get_text())
    column_marker += 1
    

Renaming  Laboratory Confirmed Cases (LCC) to Total
48700





In [4]:
df

,Locations,Total Cases,Rate,Deaths,Death Rate
0,Agoura Hills,35,167.6,0,0
1,Alhambra,186,214.47,8,9.22
2,Arcadia,80,138.52,5,8.66
3,Artesia,30,178.62,0,0
4,Avalon,0,0,0,0
...,...,...,...,...,...
339,Wiseburn,16,265.47,0,0
340,- Under Investigation,1940,,13,
341,NaN,NaN,NaN,NaN,NaN
342,NaN,NaN,NaN,NaN,NaN


In [5]:
#Add Pasadena back in 
column_marker =0
pasadenapop = 141371.
for column in soup.find_all('tr')[6].find_all('td'):
    print(column.get_text())

- Los Angeles County (excl. LB and Pas)
46212





In [6]:
#Add Pasadena back in 
column_marker =0
pasadenapop = 141371.
for column in soup.find_all('tr')[8].find_all('td'): #Previously 4, change as of May ~15
    if(column_marker <2):
        print(column.get_text())
        df.iat[-3, column_marker] = column.get_text().replace('-','').replace(' ','')
        if(column_marker ==1):
            df.iat[-3, column_marker+1] = int( column.get_text().replace('-','')) /pasadenapop *100000
    else:
        print('Skipping:', column.get_text())
    column_marker += 1
    

- Pasadena
883
Skipping: 
Skipping: 
Skipping: 


In [7]:
#Add Long Beach back in (No longer in table list like it was before)


#df.at['Long Beach',colnames[1]] = soup.find_all('tr')[7].find_all('td')[1].text.replace('*','') #Previously 3, not 7
#df.at['Long Beach',colnames[2]] = int(soup.find_all('tr')[7].find_all('td')[1].text)/longbeachpop*100000


column_marker =0
longbeachpop = 467354.
for column in soup.find_all('tr')[7].find_all('td'): #Previously 4, change as of May ~15
    if(column_marker <2):
        print(column.get_text())
        print(df.iat[-2, column_marker])
        df.iat[-2, column_marker] = column.get_text().replace('- ','')
        if(column_marker ==1):
            df.iat[-2, column_marker+1] = int( column.get_text().replace('- ','')) /longbeachpop *100000
    else:
        print('Skipping:', column.get_text())
    column_marker += 1
    

- Long Beach
nan
1605
nan
Skipping: 
Skipping: 
Skipping: 


In [8]:
df

,Locations,Total Cases,Rate,Deaths,Death Rate
0,Agoura Hills,35,167.6,0,0
1,Alhambra,186,214.47,8,9.22
2,Arcadia,80,138.52,5,8.66
3,Artesia,30,178.62,0,0
4,Avalon,0,0,0,0
...,...,...,...,...,...
339,Wiseburn,16,265.47,0,0
340,- Under Investigation,1940,,13,
341,Pasadena,883,624.598,NaN,NaN
342,Long Beach,1605,343.423,NaN,NaN


In [9]:
#Show which locations have duplicates - due to Unincorporated or City of LA
grouped = df[df['Locations'].duplicated(keep=False)].groupby('Locations')
df[df['Locations'].duplicated(keep=False)]

,Locations,Total Cases,Rate,Deaths,Death Rate
2,Arcadia,80,138.52,5,8.66
5,Azusa,159,317.75,5,9.99
11,Bradbury,3,280.64,0,0
15,Cerritos,116,231.69,1,2
16,Claremont,38,104.16,0,0
19,Covina,154,314.08,6,12.24
24,Duarte,113,513.26,14,63.59
25,El Monte,465,396.52,20,17.05
29,Glendora,138,261.54,6,11.37
31,Hawthorne,376,423.49,10,11.26


In [10]:
#Failed alt methods:
#[['Total Cases', 'Rate']].sum().head(10)
#.agg({'Total Cases' : 'sum'}).head(10)#, 'Rate' : lambda x: x.iloc[n]})

for name, group in grouped:
#    print('Before', group)
    localsum = 0
    localpop = 0
    firstindex = 0
    for row_index, row in group.iterrows():
        if firstindex==0:
            firstindex = row_index
        if (('suppressed' not in row['Total Cases']) and (int(row['Total Cases']) >0)):
            localsum = localsum + int(row['Total Cases'])
            mypop = int(row['Total Cases'])*100000/float(row['Rate'])
            localpop = localpop+mypop
            
#        print(row_index, 'I have ', row['Total Cases'], ' but my city has ', localsum)
#        print(row_index, row['Rate']) 
    if localsum >0:
        localrate = localsum*100000/localpop #This overestimates the rates in communities whose data are suppressed or have 0 confirmed cases
#        print('ROW INDEX', row_index)
        df.iat[row_index,2] = localrate
        df.iat[firstindex,2] = localrate
    else:
        df.iat[row_index,2] = 0
        df.iat[firstindex,2] = 0
    df.iat[row_index,1] = localsum
    df.iat[firstindex,1] = localsum #Note that different arguments for drop_duplicates could avoid needing to update the firstindex versions

In [11]:
#Check that it updated successfully
df[df['Locations'].duplicated(keep=False)]

,Locations,Total Cases,Rate,Deaths,Death Rate
2,Arcadia,88,133.872,5,8.66
5,Azusa,217,328.975,5,9.99
11,Bradbury,3,280.64,0,0
15,Cerritos,116,231.69,1,2
16,Claremont,38,104.16,0,0
19,Covina,210,318.902,6,12.24
24,Duarte,128,484.039,14,63.59
25,El Monte,465,396.52,20,17.05
29,Glendora,140,262.053,6,11.37
31,Hawthorne,381,417.306,10,11.26


In [12]:
df.drop_duplicates(keep='first', inplace=True)

In [13]:
df.count() 

Locations      340
Total Cases    340
Rate           340
Deaths         338
Death Rate     338
dtype: int64

In [14]:
df#.loc[df['Locations'].str.contains('Under Investigation')]

,Locations,Total Cases,Rate,Deaths,Death Rate
0,Agoura Hills,35,167.6,0,0
1,Alhambra,186,214.47,8,9.22
2,Arcadia,88,133.872,5,8.66
3,Artesia,30,178.62,0,0
4,Avalon,0,0,0,0
...,...,...,...,...,...
339,Wiseburn,16,265.47,0,0
340,- Under Investigation,1940,,13,
341,Pasadena,883,624.598,NaN,NaN
342,Long Beach,1605,343.423,NaN,NaN


In [15]:
#Other cleanup - this is different once the index has been changed:
df.loc[df['Locations'].str.contains('Under Investigation'), 'Locations'] = 'Under Investigation'
df.loc[df['Locations'].str.match('Los Angeles'), 'Locations'] = 'Los Angeles - AGGREGATE'
df.loc[df['Locations'].str.match('Laboratory Confirmed Cases (LCC)'), 'Locations'] = 'Total'

df[df['Locations'].str.contains('AGGREGATE')]

In [16]:
df.set_index('Locations', inplace=True)
df

,Total Cases,Rate,Deaths,Death Rate
Locations,,,,
Agoura Hills,35,167.6,0,0
Alhambra,186,214.47,8,9.22
Arcadia,88,133.872,5,8.66
Artesia,30,178.62,0,0
Avalon,0,0,0,0
...,...,...,...,...
Wiseburn,16,265.47,0,0
Under Investigation,1940,,13,
Pasadena,883,624.598,NaN,NaN


#Fix Long Beach and Pasadena
column_marker = 0
for column in soup.find_all('tr')[3].find_all('td'):
    print(df.loc['Long Beach'])
    df.at['Long Beach', colnames[column_marker]] = column.get_text()
    #    df.iat[df.loc['Long Beach'] , column_marker] = column.get_text()
    #    print('Before', df[df['Locations'].str.contains('Long Beach')])
    #    df.iat[df.index(df['Locations'].str.contains('Long Beach')).tolist(), column_marker] = column.get_text()
    column_marker += 1
    #    print('After', df[df['Locations'].str.contains('Long Beach')])
    

In [17]:
df.loc['Long Beach']

Total Cases       1605
Rate           343.423
Deaths             NaN
Death Rate         NaN
Name: Long Beach, dtype: object

In [18]:
df.loc['Woodland Hills']

Total Cases       155
Rate           227.76
Deaths             11
Death Rate      16.16
Name: Woodland Hills, dtype: object

In [19]:
df

,Total Cases,Rate,Deaths,Death Rate
Locations,,,,
Agoura Hills,35,167.6,0,0
Alhambra,186,214.47,8,9.22
Arcadia,88,133.872,5,8.66
Artesia,30,178.62,0,0
Avalon,0,0,0,0
...,...,...,...,...
Wiseburn,16,265.47,0,0
Under Investigation,1940,,13,
Pasadena,883,624.598,NaN,NaN


In [20]:
#Write out to CSV. Make the format match what covid_la.ipynb expects
df.index.names=['city']
df.rename(columns={'Locations': 'city', 'Total Cases': 'count', 'Rate': 'rate'}).to_csv(
    "./data/covid_"+titlestring+".csv",
    index=True,
    encoding="utf-8"
)
#THERE IS A KNOWN BUG that Under Investigation is dropped in archived scrapes! Must be readded